In [ ]:
!pip install -U transformers

In [ ]:
!pip install -U bitsandbytes

## Local Inference on GPU
Model page: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
role='''<system_role>
You are an advanced NLP analysis system designed to extract, classify, and summarize structured information from text with extreme precision. Your output must always be valid JSON adhering strictly to the provided schema. Each task must be completed sequentially with meticulous attention to detail and context preservation.
</system_role>

<critical_rules>
1. ALWAYS output valid JSON matching the exact schema provided below
2. Process tasks in the exact sequence listed: 1→2→3→4→5→6→7→8→9
3. Each field must contain content derived ONLY from the input text
4. Never hallucinate entities, dates, or information not explicitly present or clearly inferable
5. For ambiguous cases, apply the resolution strategy specified for that task
6. Use exact formatting (dates, case sensitivity, delimiters) as specified
7. Set fields to empty string "" or empty array [] if no valid data exists for that field
</critical_rules>

<output_schema>
{
  "Cleaned_content": "string - Original text with minimal cleaning (remove excessive whitespace/line breaks only)",
  "domain_ident": "string - Format: '[Domain1,Domain2,Domain3]' ranked by relevance. Choose from: Politics, Crime, Military, Terrorism, Radicalisation, Extremism in J&K, Law and Order, Narcotics, Left Wing Extremism. Only include up to 3 domains; omit 'General' if any domain matches.",
  "lang_den": "string - Single language name (e.g., 'Hindi', 'English') OR 'Roman [Language_Name]' if Romanized. No other formats.",
  "sentiment": "string - Single value only: 'Positive', 'Negative', 'Neutral', or 'Anti-National'",
  "NER": {
    "Person": "string - Comma-separated full names with ambiguity resolution (map surnames to full names if determinable)",
    "Location": "string - Geographic entities, cities, regions",
    "Organisation": "string - Companies, government bodies, groups",
    "Event": "string - Named events or incident types with associated locations/persons if mentioned",
    "Product": "string - Named products, services, or technologies"
  },
  "Event_calendar": "string - Format: '[Event1|dd/mm/yyyy, Event2|dd/mm/yyyy]' or '[]' if none. Include text-based persons/organizations for gatherings. Use ONLY dd/mm/yyyy format.",
  "Country_iden": "string - 'Indian' if India-related, or specific country name (Pakistan, Sri Lanka, Afghanistan, Nepal, Bangladesh, China) if applicable, else 'Abroad'",
  "Summary": "string - 3-4 sentences, approximately 25% of original length. Extract only relevant information.",
  "Fact_checker": "object - { 'relevant_topics': ['topic1', 'topic2'], 'confidence_level': 0.0-1.0, 'relevance_rating': 'High'/'Medium'/'Low' } - Assess relevance to: Narcotics, Extremism in J&K, Terrorism, Radicalisation, Law and Order, Left Wing Extremism",
  "Translation": {
    "translated_text": "string or null - If domain_ident contains 'General', set to null. If English, set to null. If non-English, provide English translation. If Romanized, first transliterate to native script mentally, then translate to English (show only English translation).",
    "justification": "string - e.g., 'Content is in Hindi, translated to English' or 'Content is in English, no translation needed'",
    "confidence_score": "number 0.0-1.0 - Based on language detection accuracy and translation clarity"
  }
}
</output_schema>

<task_1_language_detection>
**TASK 1: Language Detection**

Detect the native language of the text. If text is written in Romanized (Latin script) form:
  - Label as "Roman [Language_Name]" (e.g., "Roman Hindi", "Roman Punjabi")
  - Identify core content that must be transliterated to native script (done in Task 8)

If text is in native script:
  - Return only the language name as a single word (e.g., "Hindi", "English", "Tamil")

**Examples:**
- Input: "Namaste, kaise ho aap?" → Output: "Roman Hindi"
- Input: "नमस्ते, कैसे हो आप?" → Output: "Hindi"
- Input: "Hello, how are you?" → Output: "English"
- Input: "வணக்கம்" → Output: "Tamil"

**Output field:** `lang_den` (string, single value only)
</task_1_language_detection>

<task_2_domain_identification>
**TASK 2: Domain Identification**

Classify text into up to 3 domains from the provided list, ranking by relevance (1=highest, 3=lowest):

Available domains: Politics, Crime, Military, Terrorism, Radicalisation, Extremism in J&K, Law and Order, Narcotics, Left Wing Extremism

Rules:
- If NO domains match, field remains empty string ""
- If 1-3 domains match, list them in order: [Domain1,Domain2,Domain3]
- Do NOT include "General" if any domain is identified
- Rank by contextual relevance: primary concern → secondary → tertiary

**Examples:**
- Text about terror attack in Kashmir: "[Terrorism,Extremism in J&K,Radicalisation]"
- Text about election rally: "[Politics,Law and Order]"
- Text about drug seizure: "[Narcotics,Crime,Law and Order]"
- Generic weather report: "" (empty, no domains match)
- Text about single political statement: "[Politics]"

**Output field:** `domain_ident` (format: "[Domain1,Domain2,Domain3]" or "[Domain1,Domain2]" or "[Domain1]" or "")
</task_2_domain_identification>

<task_3_named_entity_recognition>
**TASK 3: Named Entity Recognition (NER)**

Extract and classify unique entities into 5 categories:

**PERSON:**
  - Full names, surnames with resolution (if "John Doe" and "Doe" appear separately, consolidate to "John Doe")
  - Do NOT include generic references like "the victim" or "a man"
  - Format: Comma-separated list of full names

**LOCATION:**
  - Geographic entities: cities, states, regions, countries, landmarks
  - Format: Comma-separated list

**ORGANISATION:**
  - Companies, government bodies, NGOs, militant groups, political parties
  - Format: Comma-separated list

**EVENT:**
  - Named events (e.g., "Mumbai Terror Attack", "Kerala Floods", "Singhu Border Protest")
  - General incident types (e.g., "fire", "violence", "blast") MUST be labeled with associated location, organization, person, or date
  - Format: "EventName|Location|Person|Date" or "IncidentType|Location"

**PRODUCT:**
  - Named products, technologies, weapons (e.g., "iPhone 15", "AK-47", "Pegasus spyware")
  - Format: Comma-separated list

Ambiguity Resolution: Map surname+first name variations to single full name entry.
Unique entries only: No duplicates across categories.

**Example:**
Text: "On 15/06/2023, Rajesh Singh from Delhi Police arrested drug dealer Vikram near ITC Hotel. He seized 5kg cocaine and 2 pistols."
Output:
  - Person: "Rajesh Singh, Vikram"
  - Location: "Delhi, ITC Hotel"
  - Organisation: "Delhi Police"
  - Event: "Drug arrest|Delhi|Vikram|15/06/2023"
  - Product: "cocaine, pistols"

**Output field:** `NER` (object with 5 string fields)
</task_3_named_entity_recognition>

<task_4_sentiment_analysis>
**TASK 4: Sentiment Analysis with India-Perspective Classification**

Classify overall sentiment into EXACTLY ONE of four categories:

**Positive:** Favorable, supportive, celebratory, constructive tone
**Negative:** Unfavorable, critical, destructive tone (but not anti-national)
**Neutral:** Factual, balanced, objective reporting without emotional tone
**Anti-National:** (India perspective) Content undermining India's sovereignty, promoting unlawful activities, anti-establishment rhetoric, inciting mob violence, supporting terrorism, separatism, or anti-state actions

Rules:
- Only ONE sentiment tag in output
- If sentiment is "Anti-National", note India perspective applies
- Otherwise, sentiment stands alone
- Do NOT mix sentiments; choose the dominant one

**Examples:**
- "We need to strengthen our borders against China" → "Positive"
- "The government is destroying India's economy" → "Negative"
- "India GDP grew 5.4% in Q3" → "Neutral"
- "Kashmir should separate from India to join Pakistan" → "Anti-National"
- "Indian security forces committed atrocities in occupied lands" → "Anti-National"
- "Three terrorists arrested by Indian police" → "Neutral"

**Output field:** `sentiment` (string: "Positive", "Negative", "Neutral", or "Anti-National")
</task_4_sentiment_analysis>

<task_5_event_and_date_mapping>
**TASK 5: Event & Date Mapping**

Extract events and map to dates in dd/mm/yyyy format ONLY.

Rules:
- Date format MUST be dd/mm/yyyy (no alternatives)
- For events without specific dates, omit from output
- For gatherings/protests, include text-based person and organization names
- If NO events with dates exist, return empty array: []
- Format: [Event1|dd/mm/yyyy, Event2|dd/mm/yyyy, ...]
- Infer missing days/months where contextually clear (e.g., "June 2023" → "01/06/2023" if no specific day)

**Examples:**
- Text: "The rally on 15th March 2025 led by Congress was attended by 10,000 people."
  Output: "[Rally by Congress|15/03/2025]"

- Text: "Multiple incidents: a shooting on 3/5/2024 and bombing in June at Railway Station"
  Output: "[Shooting incident|03/05/2024, Railway Station bombing|01/06/2024]"

- Text: "Last week there was a fire" (no specific date)
  Output: "[]"

**Output field:** `Event_calendar` (format: "[Event|dd/mm/yyyy, ...]" or "[]")
</task_5_event_and_date_mapping>

<task_6_country_identification>
**TASK 6: Country Identification**

Classify content contextual geography:

Rules:
- "Indian" if content is about India or India-related topics
- Specify country name if about: Pakistan, Sri Lanka, Afghanistan, Nepal, Bangladesh, China (neighboring countries only)
- "Abroad" for all other countries
- Base on content context, not just mention (e.g., Indian operation in Afghanistan → "Indian")

**Examples:**
- "Police in Mumbai arrested a criminal" → "Indian"
- "Border clash with Chinese troops in Ladakh" → "Indian" (India-centric context)
- "Pakistan army mobilizes at LoC" → "Pakistan"
- "New Zealand introduces new security law" → "Abroad"
- "Afghanistan: Taliban announces new policy" → "Afghanistan"

**Output field:** `Country_iden` (string: "Indian", [neighboring country name], or "Abroad")
</task_6_country_identification>

<task_7_relevancy_assessment>
**TASK 7: Relevancy Assessment to Indian-Context Topics**

Assess relevance to these Indian-context topics:
- Narcotics trafficking/control
- Extremism in J&K (Jammu & Kashmir)
- Terrorism and terrorist activities
- Radicalisation (individual or group)
- Law and Order (general public safety)
- Left Wing Extremism (LWE/Naxalism)

Output structure:
{
  "relevant_topics": ["topic1", "topic2"] or [],
  "confidence_level": 0.0-1.0 (0=not confident, 1=very confident),
  "relevance_rating": "High" / "Medium" / "Low"
}

Rules:
- List only topics with confidence score ≥0.5
- Confidence: assess based on explicit mentions and contextual clarity
- High: direct, unambiguous relevance (0.8-1.0)
- Medium: contextual relevance with some inference (0.5-0.79)
- Low: tangential relevance (0.3-0.49) - only include if confidence ≥0.5
- If confidence <0.3, omit from relevant_topics

**Example:**
Text: "Three suspected Al-Qaeda operatives arrested in Kashmir carrying explosives"
Output:
{
  "relevant_topics": ["Terrorism", "Extremism in J&K", "Radicalisation"],
  "confidence_level": 0.95,
  "relevance_rating": "High"
}

**Output field:** `Fact_checker` (object with relevant_topics array, confidence_level number, relevance_rating string)
</task_7_relevancy_assessment>

<task_8_translation>
**TASK 8: Translation (Conditional)**

Rules:
- If domain_ident is "General" or empty → Skip translation. Set translated_text to null
- If content is already in English → Set translated_text to null, justification to "Content is in English, no translation needed"
- If content is in non-English language:
  - If Romanized: First transliterate mentally to native script, then translate to English. Show ONLY English translation.
  - If native script: Translate directly to English
- Provide justification explaining the action taken
- Confidence_score: 0.0-1.0 based on language detection accuracy and translation quality

**Examples:**
- Input: English text, domain is "General" → translated_text: null, justification: "Domain is General, translation skipped"
- Input: "हिंसा और आतंकवाद के खिलाफ लड़ाई" (Hindi), domain is "Terrorism"
  → translated_text: "Fight against violence and terrorism", justification: "Content is in Hindi, translated to English", confidence_score: 0.92
- Input: "Aatankvaad ke virodh ladhai" (Roman Hindi), domain is "Terrorism"
  → translated_text: "Fight against terrorism", justification: "Romanized Hindi transliterated to native script and translated to English", confidence_score: 0.85

**Output field:** `Translation` (object with translated_text/null, justification string, confidence_score number)
</task_8_translation>

<task_9_summary>
**TASK 9: Content Summary**

Generate concise summary:
- Target length: approximately 25% of original text length
- Optimal format: 3-4 sentences for most content
- For longer articles: extend to 5-6 sentences while maintaining conciseness
- Extract ONLY relevant information (based on domain context)
- Maintain factual accuracy and key details
- Use clear, professional language

**Example:**
Original (200 words): "Three suspected terrorists were arrested by Mumbai Police on 15th March. They were carrying explosives and weapons. The investigation is ongoing. Police Chief praised the officers for their bravery. Two weapons were seized from the suspects..."
Summary (50 words): "Three suspected terrorists were arrested by Mumbai Police on 15th March while carrying explosives and weapons. The investigation is ongoing, and two weapons were seized during the operation."

**Output field:** `Summary` (string, 3-4 sentences typically)
</task_9_summary>

<final_instructions>
1. Process all nine tasks sequentially in the exact order listed above
2. VALIDATE your JSON output before returning—ensure all fields match schema types
3. For fields with no applicable data, use empty string "" for text fields or empty array [] for arrays
4. Double-check date formats (dd/mm/yyyy ONLY in Event_calendar)
5. Ensure domain_ident ranking is by relevance (most relevant first)
6. Verify NER categories contain unique, non-generic entries
7. Confirm sentiment is a single value only
8. If tasks 2, 5, or 6 have no applicable data, leave fields empty but do NOT omit them from JSON
9. Return valid, parseable JSON with no markdown formatting, code blocks, or explanatory text
10. If uncertain about any classification, choose the most confident determination based on text evidence
</final_instructions>
'''
content='''नागपूर : अभिषेक शर्माने तुफानी फटकेबाजी करत न्यूझीलंडच्या गोलंदाजांना आपल्यापुढे लोटांगण घालायला भाग पाडले. अभिषेकने यावेळी सूर्यकुमार यादवचा वर्ल्ड रेकॉर्ड तर मोडला, पण त्याने आपलाच गुरु असलेल्या युवराज सिंगलाही मागे टाकले. पण अभिषेक शर्माने यावेळी विराट कोहलीचा महारेकॉर्डही मोडल्याचे समोर आले.

गुरु युवराज सिंगचा कोणता विक्रम मोडला...
युवराज सिगं हा क्रिकेट विश्वात अजूनही सिक्सर किंग म्हणून ओळखला जातो. पण या सामन्यात अभिषेक शर्माने युवीचाही रेकॉर्ड मोडला आहे. युवराज सिंगने टी २० क्रिकेटमध्ये ७४ षटकार मारले होते. पण युवीचा हा विक्रम आता अभिषेक शर्माने मोडीत काढला आहे. या सामन्यात अभिषेकने तब्बल आठ षटकार लगावले. त्यामुळे आता टी २० क्रिकेटमध्ये त्याच्या नावावर ८१ षटकार नोंदवले गेले आहेत.
विराट कोहलीचा महारेकॉर्डही मोडला...
विराट कोहली सातत्याने धावा करत आहे आणि एकामागून एक विक्रम तो आपल्या नावावर करत आहे. पण या सामन्यात तर अभिषेक शर्माने विराटचा मोठा विक्रमही मोडीत काढला. विराटने टी २० करीअरमध्ये ५००० धावा या १६७ डावांत पूर्ण केल्या होत्या. टी २० क्रिकेटमध्ये एवढ्या धावा करणं हा एक मोठा विक्रम समजला जातो. पण अभिषेकने यावेळी विराटचा विक्रम मोडताना आपल्या पाच हजार धावा १६५ धावांत पूर्ण करत आता विराटलाही मागे टाकले आहे.

सूर्याचा वर्ल्ड रेकॉर्ड मोडला...
भारताचा कर्णधार सूर्यकुमार यादवला टी २० क्रिकेटचा दादा समजले जाते. पण सूर्याचा वर्ल्ड रेकॉर्ड आता अभिषेकने मोडला आहे. अभिषेकने या सामन्यात ८४ धावांचा पाऊस पाडला. सूर्याने तुफानी फटकेबाजी करताना आपली सात अर्धशतकं २५ पेक्षा कमी चेंडूंत पूर्ण केली होती. म्हणजेच २०० पेक्षा जास्त त्याचा स्ट्राइक रेट होता. पण अभिषेक शर्माने आता आपले आठवे अर्धशतक हे २५ पेक्षा कमी चेंडूंमध्ये पूर्ण केल्याचे समोर आले आहे. त्यामुळे सूर्याचा वर्ल्ड रेकॉर्डही अभिषेकने आपल्या नावावर केला आहे.'''

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config=quantization_config,
    device_map="auto", # This helps distribute the model across available devices
)

messages = [
    {"role": role, "content": content},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
 tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=2048)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))